<a href="https://colab.research.google.com/github/bogus1aw/text-classification-benchmark/blob/main/M_herBERT_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HerBERT benchmark for wiki dataset 

In [1]:
# check available GPU
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla P4, 418.67, 7611 MiB


In [3]:
%%capture
!pip install datasets transformers

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.is_available()

True

In [5]:
import pandas as pd

def load_corpora_to_dataframe(corpora):
  data = open(corpora).read()
  labels, texts = [], []
  for i, line in enumerate(data.split("\n")):
      content = line.split()
      labels.append(content[0])
      texts.append(" ".join(content[1:]))
  return texts, labels
  # # create a dataframe using texts and labels
  # trainDF = pd.DataFrame()
  # trainDF['text'] = texts
  # trainDF['label'] = labels
  # return trainDF

#raw    
corpora = '/content/drive/MyDrive/master_datasets/wiki_preprocessed/wikiInOneFileDataset.txt'
#lemmas
# corpora = '/content/drive/MyDrive/master_datasets/wiki_preprocessed/wikiInOneFileDataset_lemmas.txt'

texts, labels = load_corpora_to_dataframe(corpora)

In [5]:
from sklearn.model_selection import train_test_split
# create test dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2)
# create train and validation dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [6]:
print(len(train_texts), ' ', len(train_labels))
print(len(val_texts), ' ', len(val_labels))
print(len(test_texts), ' ', len(test_labels))

4406   4406
1102   1102
1377   1377


In [4]:
import torch
from transformers import HerbertTokenizer, RobertaForSequenceClassification, EvalPrediction

tokenizer = HerbertTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-klej-cased-v1", num_labels=34)

# encoded_input = tokenizer.encode("Kto ma lepszą sztukę, ma lepszy rząd – to jasne.", return_tensors='pt')
# outputs = model(encoded_input)

Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream

In [8]:
max_length = 200
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)

from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
train_labels = encoder.fit_transform(train_labels)
val_labels = encoder.fit_transform(val_labels)
test_labels = encoder.fit_transform(test_labels)

In [9]:
# build pyTorch dataset
import torch

class wikiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = wikiDataset(train_encodings, train_labels)
val_dataset = wikiDataset(val_encodings, val_labels)
test_dataset = wikiDataset(test_encodings, test_labels)

In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy = "epoch",
    num_train_epochs=10,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

from datasets import load_metric
import numpy as np
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    print(predictions[:10])
    print(labels[:10])
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.781200,0.548217,0.900181,14.132500,77.976000
2,0.319500,0.363583,0.910163,14.088000,78.222000
3,0.196800,0.353149,0.925590,14.130200,77.989000
4,0.013100,0.408879,0.926497,14.145200,77.907000
5,0.060200,0.485672,0.923775,14.149900,77.881000
6,0.012900,0.472525,0.923775,14.198200,77.615000
7,0.045900,0.492332,0.926497,14.175600,77.739000
8,0.001700,0.486194,0.928312,14.263400,77.261000
9,0.001700,0.494063,0.929220,14.207700,77.564000
10,0.001400,0.494927,0.929220,14.223500,77.477000


[30 10 27  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]


TrainOutput(global_step=2760, training_loss=0.30656987773322003, metrics={'train_runtime': 2066.6349, 'train_samples_per_second': 1.336, 'total_flos': 6603624926736000, 'epoch': 10.0})

In [11]:
trainer.evaluate()

[30 10 32  7  6 32 23 28  0 13]
[30 10 32  7  6 32 23 28  0 13]


{'epoch': 10.0,
 'eval_accuracy': 0.9292196007259528,
 'eval_loss': 0.4940628707408905,
 'eval_runtime': 14.1647,
 'eval_samples_per_second': 77.799}

In [12]:
trainer.predict(test_dataset)

[29 16 25 26  3 17 11 10  4  5]
[29 16 25 26  3 17 29 10  4  5]


PredictionOutput(predictions=array([[ 0.52533567, -0.16830575, -0.44679517, ..., -1.0219253 ,
        -0.6416591 ,  1.0066499 ],
       [-0.86272174, -1.1338904 ,  0.1913147 , ..., -0.74480337,
        -0.6725304 , -0.62956136],
       [-0.91379255, -0.5882427 , -0.9381301 , ..., -1.2275659 ,
         0.09151111, -0.17658563],
       ...,
       [ 0.41357136,  0.0255044 , -0.30955365, ..., -0.49017477,
        -0.31047082, -1.0729183 ],
       [-0.16920969,  0.2563739 ,  0.04625836, ...,  0.38938782,
        -0.53634894, -1.010129  ],
       [-0.02859521, -0.2705189 , -0.9707644 , ..., -1.0110743 ,
        -0.44467172, -0.83472806]], dtype=float32), label_ids=array([29, 16, 25, ..., 15, 18, 12]), metrics={'eval_loss': 0.400635302066803, 'eval_accuracy': 0.9419026870007262, 'eval_runtime': 17.9486, 'eval_samples_per_second': 76.719})

In [13]:
# from torch.utils.data import DataLoader
# from transformers import  AdamW

# # device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

# model.to(device)
# model.train()

# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

# model.eval()